In [20]:
# ==========================================
# Autonomous Drone Navigation via Approx Q-Learning
# ==========================================

import gymnasium as gym
from gym import spaces
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
import imageio
import os
from collections import deque
from base64 import b64encode
from IPython.display import HTML

# --- Part 1: Asset Management & Synthetic Data Generation ---
# We create dummy assets so the code runs immediately.
# Users can replace 'town_map.png' and 'drone.png' with their own files.

MAP_FILENAME = 'town_map.png'
DRONE_FILENAME = 'drone.png'

def create_dummy_assets():
    """Creates a synthetic map and drone sprite if files are missing."""
    if not os.path.exists(MAP_FILENAME):
        print("Creating dummy map...")
        # White background (Obstacles), Black Roads (Navigable)
        # 600x600 Map
        img = np.ones((600, 600, 3), dtype=np.uint8) * 255

        # Draw a complex road network (Black lines)
        # Main Loop
        pts = np.array([[50, 50], [550, 50], [550, 550], [50, 550]], np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(img, [pts], True, (0,0,0), thickness=60)

        # Cross roads
        cv2.line(img, (300, 100), (300, 500), (0,0,0), 50)
        cv2.line(img, (100, 300), (500, 300), (0,0,0), 50)

        # Diagonal shortcut
        cv2.line(img, (100, 100), (500, 500), (0,0,0), 40)

        cv2.imwrite(MAP_FILENAME, img)

    if not os.path.exists(DRONE_FILENAME):
        print("Creating dummy drone...")
        # Create a 64x64 transparent drone sprite
        # OpenCV uses BGRA for transparency
        drone = np.zeros((64, 64, 4), dtype=np.uint8)

        # Draw Blue Body
        cv2.circle(drone, (32, 32), 10, (255, 0, 0, 255), -1)
        # Draw Rotors (Red)
        cv2.circle(drone, (10, 10), 8, (0, 0, 255, 255), -1)
        cv2.circle(drone, (54, 10), 8, (0, 0, 255, 255), -1)
        cv2.circle(drone, (54, 54), 8, (0, 0, 255, 255), -1)
        cv2.circle(drone, (10, 54), 8, (0, 0, 255, 255), -1)
        # Arms
        cv2.line(drone, (10,10), (54,54), (100,100,100,255), 2)
        cv2.line(drone, (54,10), (10,54), (100,100,100,255), 2)

        cv2.imwrite(DRONE_FILENAME, drone)

create_dummy_assets()

# --- Part 2: The RL Environment ---

class DroneDeliveryEnv(gym.Env):
    metadata = {'render.modes': ['rgb_array']}

    def __init__(self, map_path, drone_path):
        super(DroneDeliveryEnv, self).__init__()

        # 1. Image Processing Setup
        self.original_map = cv2.imread(map_path)
        if self.original_map is None:
            raise ValueError("Map image not found!")

        # User Spec: Road is BLACK. Obstacles are Colored/White.
        # Convert to Grayscale
        gray = cv2.cvtColor(self.original_map, cv2.COLOR_BGR2GRAY)

        # Thresholding: Black (<50) becomes 1 (Road), else 0 (Obstacle)
        # We invert standard thresholding so Road is the "foreground"
        _, self.road_mask = cv2.threshold(gray, 60, 1, cv2.THRESH_BINARY_INV)

        self.h, self.w = self.road_mask.shape

        # 2. Auto-Scale Drone
        # Use Distance Transform to find max road width
        dist_transform = cv2.distanceTransform(self.road_mask, cv2.DIST_L2, 5)
        max_dist = np.max(dist_transform)
        road_width = max_dist * 2

        # Scale drone to 70% of road width
        self.drone_size = int(road_width * 0.7)
        if self.drone_size < 10: self.drone_size = 10 # Min size safety

        raw_drone = cv2.imread(drone_path, cv2.IMREAD_UNCHANGED)

        # Check if image is grayscale (2 dims) or RGB (3 channels)
        if len(raw_drone.shape) == 2:
            raw_drone = cv2.cvtColor(raw_drone, cv2.COLOR_GRAY2BGRA)
        elif raw_drone.shape[2] == 3:
            raw_drone = cv2.cvtColor(raw_drone, cv2.COLOR_BGR2BGRA)

        self.drone_sprite = cv2.resize(raw_drone, (self.drone_size, self.drone_size))

        # 3. Action Space (8-way movement)
        # 0:N, 1:NE, 2:E, 3:SE, 4:S, 5:SW, 6:W, 7:NW
        self.action_space = spaces.Discrete(8)
        self.moves = [(-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1)]
        self.speed = int(road_width * 0.3) # Move 30% of road width per step

        # 4. State Initialization
        self.start_pos = self._get_random_road_point()
        self.goal_pos = self._get_random_road_point()
        self.agent_pos = np.array(self.start_pos, dtype=float)

        self.max_steps = 500
        self.step_count = 0
        self.trajectory = [] # For rendering path

    def _get_random_road_point(self):
        # Return (y, x)
        ys, xs = np.where(self.road_mask == 1)
        idx = np.random.randint(len(ys))
        return np.array([ys[idx], xs[idx]])

    def reset(self):
        # Start at fixed point or random? User asked for fixed start, random goal.
        # Let's fix start to the first valid point we found initially, or randomize it.
        # For robustness, we randomize start and goal every time to train generally.
        self.agent_pos = np.array(self.start_pos, dtype=float)

        # Ensure goal is far enough
        while True:
            self.goal_pos = self._get_random_road_point()
            if np.linalg.norm(self.goal_pos - self.agent_pos) > 100:
                break

        self.step_count = 0
        self.trajectory = [tuple(self.agent_pos.astype(int))]
        return self.agent_pos

    def step(self, action):
        self.step_count += 1
        dy, dx = self.moves[action]

        # Proposed move
        new_pos = self.agent_pos + np.array([dy, dx]) * self.speed
        ny, nx = new_pos.astype(int)

        reward = -1 # Living penalty
        done = False
        info = {}

        # Check Bounds
        if not (0 <= ny < self.h and 0 <= nx < self.w):
            reward = -100
            done = True
            info['outcome'] = 'crash_boundary'
            return self.agent_pos, reward, done, info

        # Check Collision (Center point check for efficiency)
        if self.road_mask[ny, nx] == 0:
            reward = -100
            done = True
            info['outcome'] = 'crash_wall'
            return self.agent_pos, reward, done, info

        # Valid Move
        self.agent_pos = new_pos
        self.trajectory.append((ny, nx))

        # Check Goal
        dist = np.linalg.norm(self.agent_pos - self.goal_pos)
        if dist < self.drone_size: # Reached within 1 body length
            reward = 100
            done = True
            info['outcome'] = 'success'

        if self.step_count >= self.max_steps:
            done = True
            info['outcome'] = 'timeout'

        return self.agent_pos, reward, done, info

    def render(self, mode='rgb_array'):
        canvas = self.original_map.copy()

        # 1. Draw Trajectory (Red line)
        if len(self.trajectory) > 1:
            pts = np.array([self.trajectory], np.int32)
            # Note: OpenCV expects (x,y) for points, not (y,x)
            # We must swap columns for polylines
            swapped_pts = np.flip(pts, axis=2)
            cv2.polylines(canvas, swapped_pts, False, (0,0,255), 2)

        # 2. Draw Goal (Green Circle)
        gy, gx = self.goal_pos
        cv2.circle(canvas, (gx, gy), 15, (0, 255, 0), -1)
        cv2.putText(canvas, "GOAL", (gx, gy-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        # 3. Draw Drone (Alpha Blend)
        dy, dx = self.agent_pos.astype(int)
        h, w = self.drone_sprite.shape[:2]

        # ROI coordinates
        y1, y2 = dy - h//2, dy + h//2
        x1, x2 = dx - w//2, dx + w//2

        # Clip logic
        y1_c, y2_c = max(0, y1), min(self.h, y2)
        x1_c, x2_c = max(0, x1), min(self.w, x2)

        # 1. Calculate the exact dimensions of the visible background area
        bg_h = y2_c - y1_c
        bg_w = x2_c - x1_c

        # 2. Calculate start offsets into the sprite
        sy1 = y1_c - y1
        sx1 = x1_c - x1

        # 3. Calculate end offsets using the background dimensions (Guarantees match)
        sy2 = sy1 + bg_h
        sx2 = sx1 + bg_w

        if y2_c > y1_c and x2_c > x1_c:
            sprite_crop = self.drone_sprite[sy1:sy2, sx1:sx2]
            bg_crop = canvas[y1_c:y2_c, x1_c:x2_c]

            # Blend
            alpha = sprite_crop[:, :, 3] / 255.0
            inv_alpha = 1.0 - alpha

            for c in range(3):
                bg_crop[:, :, c] = (alpha * sprite_crop[:, :, c] +
                                    inv_alpha * bg_crop[:, :, c])

            canvas[y1_c:y2_c, x1_c:x2_c] = bg_crop

        return canvas

# --- Part 3: Approximate Q-Learning Agent ---

class FeatureExtractor:
    def __init__(self, env):
        self.env = env
        # Precompute distance transform for "Safety" feature
        # This map tells us how far every pixel is from a wall
        self.dist_map = cv2.distanceTransform(env.road_mask, cv2.DIST_L2, 3)
        self.dist_map = self.dist_map / np.max(self.dist_map) # Normalize 0-1
        self.max_dist = np.sqrt(env.h**2 + env.w**2)

    def get_features(self, state, action):
        # state is (y,x)
        dy, dx = self.env.moves[action]
        next_pos = state + np.array([dy, dx]) * self.env.speed
        ny, nx = next_pos.astype(int)

        # Feature 1: Bias
        f_bias = 1.0

        # Check bounds/collision for lookahead features
        valid_pos = (0 <= ny < self.env.h) and (0 <= nx < self.env.w)
        is_wall = False
        if valid_pos:
            if self.env.road_mask[ny, nx] == 0:
                is_wall = True
        else:
            is_wall = True # Out of bounds is a wall

        # Feature 2: Collision Ahead (Binary)
        f_crash = 1.0 if is_wall else 0.0

        # Feature 3: Normalized Distance to Goal
        # If we crash, distance is effectively infinite/max
        if is_wall:
            dist = self.max_dist
        else:
            dist = np.linalg.norm(next_pos - self.env.goal_pos)

        f_dist = dist / self.max_dist

        # Feature 4: Road Safety (Center-ness)
        # We want to maximize this, so if using cost minimization, invert.
        # Here we maximize Reward, so positive weight on safety.
        f_safety = 0.0
        if valid_pos and not is_wall:
            f_safety = self.dist_map[ny, nx]

        return np.array([f_bias, f_dist, f_crash, f_safety])

class LinearQAgent:
    def __init__(self, env, alpha=0.01, gamma=0.95, epsilon=1.0):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.fe = FeatureExtractor(env)

        # Weights:
        # Heuristic Init:
        # Dist: Negative (want small dist)
        # Crash: Large Negative (avoid death)
        # Safety: Positive (stay in middle)
        self.w = np.array([0.0, -1.0, -5.0, 0.5])

    def get_q(self, state, action):
        feats = self.fe.get_features(state, action)
        return np.dot(self.w, feats)

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return self.env.action_space.sample()

        q_vals = [self.get_q(state, a) for a in range(8)]
        return np.argmax(q_vals)

    def update(self, state, action, reward, next_state, done):
        q_pred = self.get_q(state, action)

        if done:
            target = reward
        else:
            q_next = [self.get_q(next_state, a) for a in range(8)]
            target = reward + self.gamma * np.max(q_next)

        td_error = target - q_pred

        # Gradient update: w += alpha * error * features
        feats = self.fe.get_features(state, action)
        self.w += self.alpha * td_error * feats

# --- Part 4: Training & Video Rendering ---

def run_simulation():
    env = DroneDeliveryEnv(MAP_FILENAME, DRONE_FILENAME)
    agent = LinearQAgent(env)

    print("Training Agent (Approx 500 episodes)...")

    for ep in range(500):
        state = env.reset()
        done = False
        total_rew = 0

        while not done:
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.update(state, action, reward, next_state, done)
            state = next_state
            total_rew += reward

        # Decay Epsilon
        agent.epsilon = max(0.01, agent.epsilon * 0.99)

        if ep % 50 == 0:
            print(f"Ep {ep}: Reward={total_rew:.1f} Epsilon={agent.epsilon:.2f} Weights={agent.w}")

    # --- Render Final Video ---
    print("\nGenerating video of trained agent...")
    frames = []
    state = env.reset()
    agent.epsilon = 0.0 # Greedy mode
    done = False

    # Limit video length
    for _ in range(400):
        frame = env.render()
        frames.append(frame)

        action = agent.act(state)
        state, r, done, info = env.step(action)
        if done:
            # Add a few freeze frames at end
            for _ in range(10): frames.append(env.render())
            print(f"Final Run Result: {info['outcome']}")
            break

    # Save MP4
    video_path = 'drone_delivery_demo.mp4'
    imageio.mimsave(video_path, frames, fps=30)

    # Show in Notebook
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
    <video width=600 controls autoplay loop>
          <source src="{data_url}" type="video/mp4">
    </video>
    """))
    print(f"Video saved to {video_path}")

if __name__ == "__main__":
    run_simulation()

Training Agent (Approx 500 episodes)...
Ep 0: Reward=-101.0 Epsilon=0.99 Weights=[-0.94954469 -1.94125003 -5.93988985  0.4957128 ]
Ep 50: Reward=-103.0 Epsilon=0.60 Weights=[-20.84576527 -25.13342967 -29.69693059   1.2447256 ]
Ep 100: Reward=-100.0 Epsilon=0.36 Weights=[-19.25385961 -31.74281253 -37.53241881   4.11399957]
Ep 150: Reward=-370.0 Epsilon=0.22 Weights=[ -7.30072052 -33.1146438  -44.55720839   7.14072843]
Ep 200: Reward=86.0 Epsilon=0.13 Weights=[  3.77579549 -36.67629722 -49.61251253   4.60117917]
Ep 250: Reward=37.0 Epsilon=0.08 Weights=[ 12.79011979 -41.74729039 -53.18578176   4.57718291]
Ep 300: Reward=-219.0 Epsilon=0.05 Weights=[ 19.53521722 -48.25106345 -56.15077639   3.39590224]
Ep 350: Reward=95.0 Epsilon=0.03 Weights=[ 26.98366502 -52.7533782  -58.1691343    5.23221442]
Ep 400: Reward=-114.0 Epsilon=0.02 Weights=[ 29.99247899 -57.68495786 -59.90670687   5.66812241]
Ep 450: Reward=-297.0 Epsilon=0.01 Weights=[ 11.98510254 -63.04690651 -59.21888164   2.69154646]



Generating video of trained agent...
Final Run Result: success


Video saved to drone_delivery_demo.mp4


In [21]:
# ==========================================
# Autonomous Drone Navigation via Approx Q-Learning (Loop Fixed)
# ==========================================

import gymnasium as gym
from gym import spaces
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
import imageio
import os
from base64 import b64encode
from IPython.display import HTML, display

# --- User Config ---
MAP_FILENAME = 'city_map.png'
DRONE_FILENAME = 'drone.png'

def check_assets():
    """Checks if user has uploaded assets; creates placeholders if not."""
    if not os.path.exists(MAP_FILENAME):
        print(f"⚠️ WARNING: {MAP_FILENAME} not found! Please upload your map image.")
        # Fallback: Create dummy map
        dummy = np.ones((300, 300, 3), dtype=np.uint8) * 255
        cv2.rectangle(dummy, (50,140), (250,160), (0,0,0), -1)
        cv2.imwrite(MAP_FILENAME, dummy)

    if not os.path.exists(DRONE_FILENAME):
        print(f"⚠️ WARNING: {DRONE_FILENAME} not found! Please upload your drone image.")
        # Fallback: Create dummy drone
        dummy_drone = np.zeros((20, 20, 4), dtype=np.uint8)
        dummy_drone[:] = (255, 0, 0, 255)
        cv2.imwrite(DRONE_FILENAME, dummy_drone)

check_assets()

# --- Part 2: The RL Environment ---

class DroneDeliveryEnv(gym.Env):
    metadata = {'render.modes': ['rgb_array']}

    def __init__(self, map_path, drone_path):
        super(DroneDeliveryEnv, self).__init__()

        # 1. Image Processing Setup
        self.original_map = cv2.imread(map_path)
        if self.original_map is None:
            raise ValueError("Map image failed to load.")

        # User Spec: "Road is the only one that is black"
        # Convert to Grayscale
        gray = cv2.cvtColor(self.original_map, cv2.COLOR_BGR2GRAY)

        # --- IMPROVED THRESHOLDING ---
        # Increase threshold slightly to catch dark grey roads (0-100 range)
        # Invert: Road (Dark) -> 1 (White), Obstacles (Bright) -> 0 (Black)
        _, self.road_mask = cv2.threshold(gray, 90, 1, cv2.THRESH_BINARY_INV)

        # Morphological Closing to connect gaps in roads
        kernel = np.ones((7,7), np.uint8)
        self.road_mask = cv2.morphologyEx(self.road_mask, cv2.MORPH_CLOSE, kernel)

        # DEBUG: Save the mask so the user can check what the computer sees
        cv2.imwrite("debug_road_mask.png", self.road_mask * 255)
        print("Debug: 'debug_road_mask.png' saved. Check this if the drone hits invisible walls.")

        self.h, self.w = self.road_mask.shape

        # 2. Auto-Scale Drone
        dist_transform = cv2.distanceTransform(self.road_mask, cv2.DIST_L2, 5)
        max_dist = np.max(dist_transform)
        road_width_px = max_dist * 2

        self.drone_size = int(road_width_px * 0.6)
        if self.drone_size < 8: self.drone_size = 8

        # Load and resize drone
        raw_drone = cv2.imread(drone_path, cv2.IMREAD_UNCHANGED)
        if raw_drone is None:
             raw_drone = np.zeros((32, 32, 4), dtype=np.uint8)
             cv2.circle(raw_drone, (16,16), 15, (255,0,0,255), -1)

        if len(raw_drone.shape) == 2:
            raw_drone = cv2.cvtColor(raw_drone, cv2.COLOR_GRAY2BGRA)
        elif raw_drone.shape[2] == 3:
            raw_drone = cv2.cvtColor(raw_drone, cv2.COLOR_BGR2BGRA)

        self.drone_sprite = cv2.resize(raw_drone, (self.drone_size, self.drone_size))

        # 3. Action Space (8-way)
        self.action_space = spaces.Discrete(8)
        # 0:N, 1:NE, 2:E, 3:SE, 4:S, 5:SW, 6:W, 7:NW
        self.moves = [(-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1), (-1,-1)]
        self.speed = int(road_width_px * 0.25)

        # 4. Setup Fixed Start
        self.fixed_start_pos = self._find_valid_start_point()

        # Initialize state variables
        self.agent_pos = np.array(self.fixed_start_pos, dtype=float)
        self.goal_pos = self._get_random_road_point()

        self.max_steps = 600 # Increased steps for complex maps
        self.step_count = 0
        self.trajectory = []

        # --- NEW: Visit Map for Loop Breaking ---
        self.visit_map = np.zeros((self.h, self.w), dtype=np.float32)

    def _find_valid_start_point(self):
        ys, xs = np.where(self.road_mask == 1)
        if len(ys) == 0: raise ValueError("No road pixels found!")
        # Use a consistent start point (e.g., middle-ish or top-left road)
        idx = int(len(ys) * 0.15)
        return np.array([ys[idx], xs[idx]])

    def _get_random_road_point(self):
        ys, xs = np.where(self.road_mask == 1)
        idx = np.random.randint(len(ys))
        return np.array([ys[idx], xs[idx]])

    def reset(self):
        self.agent_pos = np.array(self.fixed_start_pos, dtype=float)

        # Ensure goal is reachable and far
        attempts = 0
        while True:
            self.goal_pos = self._get_random_road_point()
            if np.linalg.norm(self.goal_pos - self.agent_pos) > (self.h * 0.25):
                break
            attempts += 1
            if attempts > 100: break # Safety break

        self.step_count = 0
        self.trajectory = [tuple(self.agent_pos.astype(int))]

        # Reset Visit Map (Decay or Zero)
        self.visit_map = np.zeros((self.h, self.w), dtype=np.float32)

        return self.agent_pos

    def step(self, action):
        self.step_count += 1
        dy, dx = self.moves[action]

        new_pos = self.agent_pos + np.array([dy, dx]) * self.speed
        ny, nx = new_pos.astype(int)

        reward = -0.1
        done = False
        info = {}

        # Bounds Check
        if not (0 <= ny < self.h and 0 <= nx < self.w):
            reward = -50
            done = True
            info['outcome'] = 'crash_boundary'
            return self.agent_pos, reward, done, info

        # Collision Check
        if self.road_mask[ny, nx] == 0:
            reward = -50
            done = True
            info['outcome'] = 'crash_wall'
            return self.agent_pos, reward, done, info

        # Valid Move
        self.agent_pos = new_pos
        self.trajectory.append((ny, nx))

        # --- UPDATE VISIT MAP ---
        # Mark this location as visited.
        # We use a small kernel to mark a 'patch' so slight jitters count as same spot
        y_min, y_max = max(0, ny-5), min(self.h, ny+5)
        x_min, x_max = max(0, nx-5), min(self.w, nx+5)
        self.visit_map[y_min:y_max, x_min:x_max] += 1.0

        # Goal Check
        dist = np.linalg.norm(self.agent_pos - self.goal_pos)
        if dist < self.drone_size:
            reward = 100
            done = True
            info['outcome'] = 'success'

        if self.step_count >= self.max_steps:
            done = True
            info['outcome'] = 'timeout'

        return self.agent_pos, reward, done, info

    def render(self, mode='rgb_array'):
        canvas = self.original_map.copy()

        # 1. Draw Path
        if len(self.trajectory) > 1:
            pts = np.array([self.trajectory], np.int32)
            swapped_pts = np.flip(pts, axis=2)
            cv2.polylines(canvas, swapped_pts, False, (255, 0, 0), 2)

        # 2. Draw Goal
        gy, gx = self.goal_pos
        cv2.circle(canvas, (gx, gy), 10, (0, 200, 0), -1)
        cv2.putText(canvas, "GOAL", (gx, gy-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,200,0), 2)

        # 3. Draw Drone
        dy, dx = self.agent_pos.astype(int)
        h, w = self.drone_sprite.shape[:2]

        y1, y2 = dy - h//2, dy + h//2
        x1, x2 = dx - w//2, dx + w//2
        #y1_c, y2_c = max(0, y1), min(self.h, y2)
        #x1_c, x2_c = max(0, x1), min(self.w, x2)

        #sy1, sy2 = y1_c - y1, h - (y2 - y2_c)
        #sx1, sx2 = x1_c - x1, w - (x2 - x2_c)

        # Clip logic
        y1_c, y2_c = max(0, y1), min(self.h, y2)
        x1_c, x2_c = max(0, x1), min(self.w, x2)

        # 1. Calculate the exact dimensions of the visible background area
        bg_h = y2_c - y1_c
        bg_w = x2_c - x1_c

        # 2. Calculate start offsets into the sprite
        sy1 = y1_c - y1
        sx1 = x1_c - x1

        # 3. Calculate end offsets using the background dimensions (Guarantees match)
        sy2 = sy1 + bg_h
        sx2 = sx1 + bg_w

        if y2_c > y1_c and x2_c > x1_c:
            sprite_crop = self.drone_sprite[sy1:sy2, sx1:sx2]
            bg_crop = canvas[y1_c:y2_c, x1_c:x2_c]
            alpha = sprite_crop[:, :, 3] / 255.0
            inv_alpha = 1.0 - alpha
            for c in range(3):
                bg_crop[:, :, c] = (alpha * sprite_crop[:, :, c] + inv_alpha * bg_crop[:, :, c])
            canvas[y1_c:y2_c, x1_c:x2_c] = bg_crop

        return canvas

# --- Part 3: Approximate Q-Learning Agent ---

class FeatureExtractor:
    def __init__(self, env):
        self.env = env
        # Safety Map
        self.dist_map = cv2.distanceTransform(env.road_mask, cv2.DIST_L2, 3)
        self.dist_map = self.dist_map / (np.max(self.dist_map) + 1e-5)
        self.max_env_dist = np.sqrt(env.h**2 + env.w**2)

    def get_features(self, state, action):
        """
        Features:
        """
        dy, dx = self.env.moves[action]
        next_pos = state + np.array([dy, dx]) * self.env.speed
        ny, nx = next_pos.astype(int)

        # 1. Bias
        f_bias = 1.0

        # 2. Check Lookahead
        in_bounds = (0 <= ny < self.env.h) and (0 <= nx < self.env.w)
        is_crash = False
        if not in_bounds:
            is_crash = True
        elif self.env.road_mask[ny, nx] == 0:
            is_crash = True

        # Feature: Collision
        f_crash = 1.0 if is_crash else 0.0

        # Feature: Normalized Distance
        if is_crash:
            dist = self.max_env_dist
        else:
            dist = np.linalg.norm(next_pos - self.env.goal_pos)
        f_dist = dist / self.max_env_dist

        # Feature: Safety
        f_safety = 0.0
        if not is_crash:
            f_safety = self.dist_map[ny, nx]

        # --- NEW Feature: Visited Count ---
        # This returns how many times we have been to 'ny, nx'
        # We clip it at 10.0 to prevent exploding values
        f_visited = 0.0
        if in_bounds:
            f_visited = min(self.env.visit_map[ny, nx], 10.0) / 10.0

        return np.array([f_bias, f_dist, f_crash, f_safety, f_visited])

class LinearQAgent:
    def __init__(self, env, alpha=0.01, gamma=0.95, epsilon=1.0):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.fe = FeatureExtractor(env)

        # Weights:
        # Visited Weight is NEGATIVE to punish backtracking
        self.w = np.array([0.0, -2.0, -10.0, 0.5, -2.5])

    def get_q(self, state, action):
        feats = self.fe.get_features(state, action)
        return np.dot(self.w, feats)

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return self.env.action_space.sample()
        q_vals = [self.get_q(state, a) for a in range(8)]
        return np.argmax(q_vals)

    def update(self, state, action, reward, next_state, done):
        q_pred = self.get_q(state, action)
        if done:
            target = reward
        else:
            q_next = [self.get_q(next_state, a) for a in range(8)]
            target = reward + self.gamma * np.max(q_next)

        td_error = target - q_pred
        feats = self.fe.get_features(state, action)
        self.w += self.alpha * td_error * feats

# --- Part 4: Execution ---

def run_simulation():
    if not os.path.exists(MAP_FILENAME):
        print("Error: Map file missing.")
        return

    env = DroneDeliveryEnv(MAP_FILENAME, DRONE_FILENAME)
    agent = LinearQAgent(env)

    print("Training Agent (Approx 3500 episodes)...")

    for ep in range(3500):
        state = env.reset()
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.update(state, action, reward, next_state, done)
            state = next_state

        # Slow decay to allow exploration of complex map
        if ep > 50: agent.epsilon = max(0.05, agent.epsilon * 0.995)
        if ep % 100 == 0:
            print(f"Ep {ep}: Epsilon={agent.epsilon:.2f} Weights={np.round(agent.w, 2)}")

    # Render Video
    print("\nGenerating final navigation video...")
    frames = []
    state = env.reset()
    agent.epsilon = 0.0 # Greedy mode
    done = False

    for _ in range(800): # Increased max frames for video
        frame = env.render()
        frames.append(frame)

        action = agent.act(state)
        state, _, done, info = env.step(action)
        if done:
            print(f"Outcome: {info['outcome']}")
            for _ in range(20): frames.append(env.render())
            break

    # Save
    video_path = 'drone_mission_fixed.mp4'
    imageio.mimsave(video_path, frames, fps=30)

    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
    <h3>Mission Replay (Loop Fixed)</h3>
    <video width=600 controls autoplay loop>
          <source src="{data_url}" type="video/mp4">
    </video>
    """))

if __name__ == "__main__":
    run_simulation()

Debug: 'debug_road_mask.png' saved. Check this if the drone hits invisible walls.
Training Agent (Approx 3500 episodes)...
Ep 0: Epsilon=1.00 Weights=[ -0.38  -2.38 -10.38   0.5   -2.5 ]
Ep 100: Epsilon=0.78 Weights=[-10.81 -13.82 -22.48   1.14  -2.69]
Ep 200: Epsilon=0.47 Weights=[ -9.28 -14.43 -24.5    2.48  -2.17]
Ep 300: Epsilon=0.29 Weights=[ -5.91 -14.4  -26.72   4.68  -1.28]
Ep 400: Epsilon=0.17 Weights=[ -2.11 -14.95 -29.99   7.53   1.93]
Ep 500: Epsilon=0.10 Weights=[ -0.99 -15.93 -32.61   8.36   1.68]
Ep 600: Epsilon=0.06 Weights=[ -0.19 -15.56 -33.46   5.91  -1.48]
Ep 700: Epsilon=0.05 Weights=[  5.04 -15.11 -36.49   3.61  -1.87]
Ep 800: Epsilon=0.05 Weights=[  5.78 -14.65 -38.54   1.94  -1.08]
Ep 900: Epsilon=0.05 Weights=[  7.31 -16.68 -39.47   1.79  -1.14]
Ep 1000: Epsilon=0.05 Weights=[  7.42 -18.37 -39.3    0.44  -1.38]
Ep 1100: Epsilon=0.05 Weights=[  9.74 -19.8  -39.32   0.9   -1.17]
Ep 1200: Epsilon=0.05 Weights=[  9.52 -19.86 -38.83   0.34  -1.16]
Ep 1300: Epsilon=0